#### Load dependencies

In [ ]:
import os
import sys
sys.path.append('Documents/Work/Texas/Research/Gplates/pygplates/Plate_forces_workflow/Requirements/pygplates_rev28_python38_win64') # <- import the appropriate one for your setup
import pygplates
import numpy as np
import netCDF4 as nc

#### Get list of points and thicknesses from a raster

In [ ]:
# Load netCDF file
root_dir = 'C:/Users/edcle/Documents/Work/Texas/Research/Svalbard/2023-bundle-forEC/GPlates-files/'
cthick = nc.Dataset(root_dir+'2-Rasters/Reconstructed-crustal-thickness/80Ma_thickness_masked_exp.nc')

# Read crustal thickness file
latitude_array = cthick['y'][:]
longitude_array = cthick['x'][:]
thickness_array = cthick['z'][:]

# append values to list
points_list = []
thickness_list = []
for i in range(len(latitude_array)):
    for j in range(len(longitude_array)):
        points_list.append((latitude_array[i],longitude_array[j]))
        thickness_list.append(float(thickness_array[i,j]))

#### Thickness list only for points within topologies

In [ ]:
rift_phase_start_time = 247
previous_phase_end = 166
root_dir = 'C:/Users/edcle/Documents/Work/Texas/Research/Svalbard/2023-bundle-forEC/GPlates-files/'
cthick = nc.Dataset(root_dir+'2-Rasters/Reconstructed-crustal-thickness/'+str(previous_phase_end)+'Ma_thickness_masked.nc')

# Load crustal thickness from raster
# latitude_array = cthick['lat'][:]
# longitude_array = cthick['lon'][:]
# thickness_array = cthick['Band1'][:]

latitude_array = cthick['y'][:]
longitude_array = cthick['x'][:]
thickness_array = cthick['z'][:]

# load and reconstruct topologies
topology_features = [pygplates.FeatureCollection(root_dir+'1-Deforming-Model/1.5-deforming-mesh/deforming-mesh-updated-hinges-fixed1020101-ProfileD.gpml'),
                     pygplates.FeatureCollection(root_dir+'1-Deforming-Model/1.4-hinge-lines/MODEL-hinges-for-deforming-updatedpasttimes-ProfD.gpml'),
                     pygplates.FeatureCollection(root_dir+'1-Deforming-Model/1.1-general-other/hinge-polygons-1020101ProfD.gpml')]
rotation_model = pygplates.RotationModel(root_dir+'1-Deforming-Model/NAtlDef_CEED-MODELB1.rot')
resolved_topologies = []
pygplates.resolve_topologies(topology_features, rotation_model, resolved_topologies, rift_phase_start_time, anchor_plate_id=301)

# create array of crustal thickness for points within the topology
points_list = []
thickness_list = []
for i in range(len(latitude_array)):
    for j in range(len(longitude_array)):
        if str(float(thickness_array[i,j])) != 'nan':
            point = pygplates.PointOnSphere(latitude_array[i],longitude_array[j])
            for resolved_topology in resolved_topologies:
                if resolved_topology.get_resolved_boundary().is_point_in_polygon(point):
                    points_list.append((latitude_array[i],longitude_array[j]))
                    thickness_list.append(float(thickness_array[i,j]))

#### Make combined raster from scalar coverages and cookie-cut raster

In [ ]:
reconstruction_time = '166'
#Load in cookie-cut raster
root_dir = 'C:/Users/edcle/Documents/Work/Texas/Research/Svalbard/2023-bundle-forEC/GPlates-files/'
cthick = nc.Dataset(root_dir+'2-Rasters/Reconstructed-crustal-thickness/crustal_thickness_from_gravity_inversion_geo_'+reconstruction_time+'.00Ma.nc')
latitude_array = cthick['lat'][:]
longitude_array = cthick['lon'][:]
thickness_array = cthick['Band1'][:]

#Append values to list
points_list = []
for i in range(len(latitude_array)):
    for j in range(len(longitude_array)):
        if str(float(thickness_array[i,j])) != 'nan':
            points_list.append([longitude_array[j],latitude_array[i],float(thickness_array[i,j])])

#Load in Scalar coverages            
f = open(root_dir+"4-Scalar-coverages/scalar_coverage_"+reconstruction_time+".00Ma.xy", "r")
lines = f.readlines()
for line in lines:
    if line[0] == '>':
        continue
    cols = line.split(" ")
    while ("" in cols):
        cols.remove("")
    points_list.append(cols)
f.close()

#Write to txt file
file = open('temp_thick.txt','w')
for line in points_list:
    file.write(str(line[0])+' '+str(line[1])+' '+str(line[2])+' '+"\n")
file.close()

#create grid file
# NB make sure to change output file name
outdir = 'C:/Users/edcle/Documents/Work/Texas/Research/Svalbard/2023-bundle-forEC/GPlates-files/2-Rasters/Reconstructed-crustal-thickness/'
!gmt blockmean temp_thick.txt -I0.05 -R-7.5/20.5/60.5/74.5 > temp_blockmean.txt
!gmt surface temp_blockmean.txt -I0.05 -R-7.5/20.5/60.5/74.5 -G$outdir/{reconstruction_time}Ma_combined_surface.nc
!gmt grdmask $outdir/mask_{reconstruction_time}.00Ma.xy -G$outdir/mask_{reconstruction_time}Ma.grd -I0.05 -R-7.5/20.5/60.5/74.5 -NNaN/1/1
!gmt grdmath $outdir/{reconstruction_time}Ma_combined_surface.nc $outdir/mask_{reconstruction_time}Ma.grd OR = $outdir/{reconstruction_time}Ma_thickness_masked.nc
    
#clean up
os.remove('temp_thick.txt')
os.remove('temp_blockmean.txt')

#### Create scalar coverages for GPlates

In [ ]:
multi_point = pygplates.MultiPointOnSphere(points_list)

scalar_coverages = {
    pygplates.ScalarType.create_gpml('CrustalThickness'): thickness_list}

ct_feature = pygplates.Feature()
ct_feature.set_geometry((multi_point,scalar_coverages))
ct_feature.set_name('Crustal Thickness')

output_feature_collection = pygplates.FeatureCollection(ct_feature)
output_feature_collection.write(root_dir+'4-Scalar-coverages/166Ma_forward_thickness.gpml')